# ISA API example using tab/json conversion microservices

This notebook aims to show how to use the ISA API for loading, editing, and writing ISA tabs, using the cloud infrastructure set up using the PhenoMeNal [workflow-demo tutorial](https://github.com/phnmnl/workflow-demo). The aim of this is to: 

1. Convert an ISA tab into a ISA JSON document;
2. Edit values in the JSON objects;
3. Convert the edited ISA JSON document back into an ISA tab.

This gives us a basic framework for editing ISA tab files programmatically.

The code snippets in this notebook use Python to import the isatab2json and the json2isatab converters. Each converter is used to load from tab into some JSON objects, and them dump back to tab the modified JSON objects of ISA content. This example makes use of running microservices calls for the conversions inside a MANTL orchestrated cloud cluster.

The ISA API is documented in this [page](http://www.github.com/ISA-agents/isa-api/wiki).

## Prerequisites

There are some prerequisites to fulfill in order to successfully run the examples in this page.

Please run the following snippet and insert your MANTL control node URL.

In [2]:
control = input()

control.djcomlab.phenomenal.cloud


Please run the following code snippet and insert your MANTL admin password.

In [3]:
import getpass
password = getpass.getpass()

Run the following code snippets to get some sample input data.

N.B. You can ignore any warning about unverified HTTP requests. Response code 204 means that the REST call succeed.

In [4]:
import urllib.request
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/ISA-agents/isa-api/master/tests/data/BII-S-3/i_gilbert.txt", 
    "i_investigation.txt"
)

In [5]:
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/ISA-agents/isa-api/master/tests/data/BII-S-3/s_BII-S-3.txt", 
    "s_BII-S-3.txt"
)

In [6]:
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/ISA-agents/isa-api/master/tests/data/BII-S-3/a_gilbert-assay-Gx.txt", 
    "a_gilbert-assay-Gx.txt"
)

In [7]:
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/ISA-agents/isa-api/master/tests/data/BII-S-3/a_gilbert-assay-Tx.txt", 
    "a_gilbert-assay-Tx.txt"
)

## Step 1: isatab2json - Converting ISA tab to ISA JSON

ISA tab files are a collection of comma separated variable files, typically loaded and edited in ISAcreator. 

In this step we convert an existing ISA tab to ISA JSON. This will allow us to more easily treat ISA content as objects. Instead of using the ISA API locally, we utlise a MANTL cluster to run the conversion as a docker microservice. This is to emulate how we might deploy such microservices within a cloud infrastructure.

In [8]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning) # suppress warnings

In [ ]:
import os
import json
import time
url = "https://admin:"+password+"@"+control+"/chronos/scheduler/iso8601"
job_json = """
{ 
    "cpus": "0.25",
    "mem": "128",  
    "epsilon" : "PT10M",  
    "name" : "isatab2json",
    "container": {
        "type": "DOCKER",
        "image": "djcomlab/isatab2json",
        "volumes": [{
            "hostPath": "/mnt/container-volumes/jupyter",
            "containerPath": "/data",
            "mode": "RW"
         }]
    },
    "command" : "python run_tab2json.py /data/ /data/",
    "owner" : "david.johnson@oerc.ox.ac.uk"
}
"""
response = requests.post(url, headers={'content-type': 'application/json'}, data=job_json, verify=False)
print("HTTP response code: " + str(response.status_code))
if response.status_code == 204:
    print("Waiting for ISA tab to JSON conversion to complete...")
    for x in range(0, 30): 
        if os.path.exists('BII-S-3.json'): break
        time.sleep(1)  # Wait until the JSON is written by the microservice job
    print("Conversion finished.")

## Step 2: Edit ISA JSON

In this step we edit up the JSON. We can modify existing existing values. The example below adds a new publication to the investigation object, which corresponds to the investigation file in an ISA tab. We load the ISA JSON output from the microservice in the previous step and append some JSON to the list of publications.

In [38]:
isa_json = json.load(open('BII-S-3.json'))
isa_json['publications'].append(
    {
        "authorList": "Gilbert JA, Field D, Huang Y, Edwards R, Li W, Gilna P, Joint I.",
        "doi": "10.1371/journal.pone.0003042",
        "pubMedID": "18725995",
        "status": {
            "annotationValue": "indexed in PubMed",
            "termAccession": "",
            "termSource": ""
        },
        "title": "Detection of large numbers of novel sequences in the metatranscriptomes of complex marine microbial communities."
    }
)
json.dump(open('BII-S-3.json', 'w'))  # BUG HERE: Chronos seems to write back file as root, causes problems
print(json.dumps(isa_json['publications']))

## Step 3: json2isatab - Convert the edited JSON back to ISA tab

Once finished with editing the ISA content by directly modifying the JSON we can now write back to ISA tab format, effectively having loaded, edited, and now saving ISA tab files. Again, we utilise our cloud infrastructure to run a microservice to convert back to ISA tab.

In [40]:
url = "https://admin:"+password+"@"+control+"/chronos/scheduler/iso8601"
if not os.path.exists('out/'):
    os.mkdir('out/')
json = """
{ 
    "cpus": "0.25",
    "mem": "128",  
    "epsilon" : "PT10M",  
    "name" : "json2isatab",
    "container": {
        "type": "DOCKER",
        "image": "djcomlab/json2isatab",
        "volumes": [{
            "hostPath": "/mnt/container-volumes/jupyter",
            "containerPath": "/data",
            "mode": "RW"
         }]
    },
    "command" : "python run_json2tab.py /data/BII-S-3.json /data/out/",
    "owner" : "david.johnson@oerc.ox.ac.uk"
}
"""
response = requests.post(url, headers={'content-type': 'application/json'}, data=json, verify=False)
print("HTTP response code: " + str(response.status_code))

If all went as planned, you should have written out some ISA tab files that correspond to the edit you made to the ISA JSON in Step 2.